# Fraud Detector - Model Training and Detector Creation

---

## Setup

Create S3 bucket and upload related training data for model training.

## Implementation

Create the resoures, model and detector with boto3 python SDK

The the data set is [project_1_data](./project_1_newaccounts_100k.csv).

#### Environment Setup

In [ ]:
!pip install -U pip
!pip install boto3

Please check CloudFormation stack to copy the `FraudDetectorDataAccessRole` from Outputs tab and paste in below as value for `DATA_ACCESS_ROLE_ARN`.

In [ ]:
# Setup proper Role and copy Role ARN in below
DATA_ACCESS_ROLE_ARN = ""

### S3 bucket setup

In [ ]:
import boto3

from IPython.core.display import display, HTML
from IPython.display import clear_output
import numpy as np
np.seterr(divide='ignore', invalid='ignore')


import pandas as pd
import uuid 
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# -- dask for parallelism -- 
import dask 

# -- standard stuff -- 
import time
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
session = boto3.Session()
region = session.region_name
account_id = session.client('sts').get_caller_identity().get('Account')
bucket_name = f"{account_id}-fraud-detector-lab-{region}"
s3 = boto3.client('s3')
try:
    if ("us-east-1" == region):
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={
            'LocationConstraint': region})
except:
    pass


In [ ]:
# Upload data file into S3 bucket
s3.upload_file('./project_1_newaccounts_100k.csv', bucket_name, 'project_1_newaccounts_100k.csv')

In [ ]:
train_data_uri = f"s3://{bucket_name}/project_1_newaccounts_100k.csv"
train_data_uri

#### Explore Data

Have a quick look at the new account data file:
* Field 'EVENT_TIMESTAMP' & 'EVENT_LABEL' are two standard and necessary fields for model training.
* customer name and customer address won't be used in model training

In [ ]:
df = pd.read_csv('./project_1_newaccounts_100k.csv')
df.info()

### Create Fraud Detector Resources (Variable, label, event and event type)

In [ ]:
# -- fraud detector client --
fraud_detector = boto3.client('frauddetector')

#### Labels Creation
A label classifies an event as fraudulent or legitimate. Once you have created a label, add the label to the event type by calling the PutEventType API.

In [ ]:
# collect existing labels
response = fraud_detector.get_labels()
existing_label_names = [label['name'] for label in response['labels']]

# setup fraudulent and legitimate labels; must match with the actual label value.
LABEL_FRAUD = '1'
LABEL_LEGIT = '0'

if LABEL_FRAUD not in existing_label_names:
    fraud_detector.put_label(
        name = LABEL_FRAUD,
        description = 'label for fraud events'
    )

if LABEL_LEGIT not in existing_label_names:
    fraud_detector.put_label(
        name = LABEL_LEGIT,
        description = 'label for legit events'
    )


#### Variables creation
Variables represent data elements that you want to use in a fraud prediction, such as data from the event that is being evaluated or risk score outputs from Amazon Fraud Detector models or Amazon SageMaker models.



In [ ]:
# collect variables
response = fraud_detector.get_variables()
existing_variable_names = [variable['name'] for variable in response['variables']]


# replace with your preferred if needed
VARIABLE_IP_ADDRESS = 'ip_address'
VARIABLE_EMAIL_ADDRESS = 'email_address'
VARIABLE_PHONE_NUMBER = 'phone_number'
VARIABLE_USER_AGENT = 'user_agent'
VARIABLE_CUSTOMER_STATE = 'customer_state'
VARIABLE_CUSTOMER_POSTAL = 'customer_postal'
VARIABLE_CUSTOMER_ADDRESS = 'customer_address'
VARIABLE_CUSTOMER_CITY = 'customer_city'

event_variables = [
    VARIABLE_IP_ADDRESS,
    VARIABLE_EMAIL_ADDRESS,
    VARIABLE_PHONE_NUMBER,
    VARIABLE_USER_AGENT,
    VARIABLE_CUSTOMER_STATE,
    VARIABLE_CUSTOMER_POSTAL,
    VARIABLE_CUSTOMER_ADDRESS,
    VARIABLE_CUSTOMER_CITY
]

#Create variable ip_address
if VARIABLE_IP_ADDRESS not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_IP_ADDRESS,
        variableType = 'IP_ADDRESS',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )
    
#Create variable email_address
if VARIABLE_EMAIL_ADDRESS not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_EMAIL_ADDRESS,
        variableType = 'EMAIL_ADDRESS',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )
    
#Create variable phone_number
if VARIABLE_PHONE_NUMBER not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_PHONE_NUMBER,
        variableType = 'PHONE_NUMBER',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )    
    
#Create variable user_agent
if VARIABLE_USER_AGENT not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_USER_AGENT,
        variableType = 'USERAGENT',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )
    
#Create variable customer_state
if VARIABLE_CUSTOMER_STATE not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_CUSTOMER_STATE,
        variableType = 'BILLING_STATE',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )    
    
#Create variable customer_postal
if VARIABLE_CUSTOMER_POSTAL not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_CUSTOMER_POSTAL,
        variableType = 'BILLING_ZIP',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )    
    
#Create variable customer_address
if VARIABLE_CUSTOMER_ADDRESS not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_CUSTOMER_ADDRESS,
        variableType = 'SHIPPING_ADDRESS_L1',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )   
    
#Create variable customer_city
if VARIABLE_CUSTOMER_CITY not in existing_variable_names:
    fraud_detector.create_variable(
        name = VARIABLE_CUSTOMER_CITY,
        variableType = 'BILLING_CITY',
        dataType = 'STRING',
        dataSource = 'EVENT'
    )    

#### Entity type

An entity represents who is performing the event. As part of a fraud prediction, you can pass the entity ID to indicate the specific entity who performed the event.

An entity type classifies the entity. Example classifications include customer, merchant, or account.

In [ ]:
# collect entity types
response = fraud_detector.get_entity_types()
existing_entity_type_names = [entity_type['name'] for entity_type in response['entityTypes']]

# replace with your preferred if needed
ENTITY_TYPE_NAME = "account"

if ENTITY_TYPE_NAME not in existing_entity_type_names:
    fraud_detector.put_entity_type(
        name = ENTITY_TYPE_NAME,
        description = 'Sample account entity type'
    )


#### Event Type

With Amazon Fraud Detector, you generate fraud predictions for events. An event type defines the structure for an individual event sent to Amazon Fraud Detector. Once defined, you can build models and detectors that evaluate the risk for specific event types.

In [ ]:
# collect event types
response = fraud_detector.get_event_types()
existing_event_type_names = [event_type['name'] for event_type in response['eventTypes']]

# replace with your preferred if needed
EVENT_TYPE_NAME = "my_new_account_registration"

if EVENT_TYPE_NAME not in existing_event_type_names:
    fraud_detector.put_event_type (
        name = EVENT_TYPE_NAME,
        description='Sample event type',
        eventVariables = event_variables,
        labels = [
            LABEL_LEGIT, 
            LABEL_FRAUD
        ],
        entityTypes = [ 
            ENTITY_TYPE_NAME 
        ]
    )

#### Model
A model version is the term Amazon Fraud Detector uses for a trained fraud detection machine learning model. All steps in the model training process are fully automated including data validation, data transformation, feature engineering, algorithm selection, training, and model optimization. Creating a model requires selecting the model type and specifying the model version configuration. The model type specifies the algorithms and transformations used to build the model.

In [ ]:
# Collect model ids
response = fraud_detector.get_models()
existing_model_ids = [ model['modelId'] for model in response['models'] ]

# Create model (which is a container for model version), replace with your preferred if needed
MODEL_ID = "sample_project_x"

if MODEL_ID not in existing_model_ids:
    fraud_detector.create_model (
        modelId = MODEL_ID,
        eventTypeName = EVENT_TYPE_NAME,
        modelType = 'ONLINE_FRAUD_INSIGHTS'
    )

In [ ]:
def get_active_model_version(model_id):
    response = fraud_detector.describe_model_versions(
        modelId = model_id,
        modelType='ONLINE_FRAUD_INSIGHTS'
    )
    active_model_version = next( (model_version for model_version in response['modelVersionDetails'] if 'ACTIVE' == model_version['status']), None) 
    return active_model_version

def check_model_version_status(target_status, model_id, model_version_number):
    print(f"to check model version status till [{target_status}]")
    while True:
        response = fraud_detector.get_model_version(
            modelId = model_id,
            modelType = 'ONLINE_FRAUD_INSIGHTS',
            modelVersionNumber = model_version_number
        )
        print(f"Model version {model_id}:{model_version_number} status check: {response['status']}")
        if target_status == response['status']:
            break
        else:
            time.sleep(60)    

In [ ]:
%%time

# Create model version, which start the model training; 
# FOR THE LAB, PLEASE DON'T RUN IT MULTIPLE TIMES UNLESS IT'S NECESSARY!
# model version creation (model training) takes approx. 50-60mins with existing sample data - the 100k one.

# Assumption - if there is no active model version, train one and update it to be active.
model_version_number = None
active_model_version = get_active_model_version(MODEL_ID)
if active_model_version:
    model_version_number = active_model_version['modelVersionNumber']
    print(f"Using existing model version and the version number: {modelVersionNumber}")
else:
    response = fraud_detector.create_model_version (
        modelId = MODEL_ID,
        modelType = 'ONLINE_FRAUD_INSIGHTS',
        trainingDataSource = 'EXTERNAL_EVENTS',
        trainingDataSchema = {
            'modelVariables' : [
                VARIABLE_IP_ADDRESS,
                VARIABLE_EMAIL_ADDRESS,
                VARIABLE_PHONE_NUMBER,
                VARIABLE_USER_AGENT,
                VARIABLE_CUSTOMER_STATE,
                VARIABLE_CUSTOMER_POSTAL,
                VARIABLE_CUSTOMER_CITY
            ],
            'labelSchema' : {
                'labelMapper' : {
                    'FRAUD' : [ LABEL_FRAUD ],
                    'LEGIT' : [ LABEL_LEGIT ]
                }
            }
        }, 
        externalEventsDetail = {
            'dataLocation' : train_data_uri,
            'dataAccessRoleArn' : DATA_ACCESS_ROLE_ARN
        }
    )
    model_version_number = response['modelVersionNumber']
    print(f"Create a new model version and the version number: {model_version_number}")

    # wait until the training finish - may takes 50-60mins
    check_model_version_status('TRAINING_COMPLETE', MODEL_ID, model_version_number)
    
    # activate/deploy model version
    fraud_detector.update_model_version_status (
        modelId = MODEL_ID,
        modelType = 'ONLINE_FRAUD_INSIGHTS',
        modelVersionNumber = model_version_number,
        status = 'ACTIVE'
    )
    

#### Detector 
A detector contains the detection logic, such as the models and rules, for a particular event that you want to evaluate for fraud. Each detector can evaluate one event type.

A detector can have multiple versions, with each version having a status of DRAFT, ACTIVE, or INACTIVE. Only one detector version can be in ACTIVE status at a time.

In [ ]:
# Collect existing detector ids
response = fraud_detector.get_detectors()
existing_detector_ids = [detector['detectorId'] for detector in response['detectors']]

# replace with your preferred if needed
DETECTOR_ID = "my_new_account_fraud_detector"

# Detector is a container for detector versions.
if DETECTOR_ID not in existing_detector_ids:
    fraud_detector.put_detector (
        detectorId = DETECTOR_ID,
        eventTypeName = EVENT_TYPE_NAME
    )

##### Create Outcome
An outcome is the result of a fraud prediction. Create an outcome for each possible fraud prediction result. For example, you may want outcomes to represent risk levels (high_risk, medium_risk, and low_risk) or actions (approve, review). Once created, you can add one or more outcomes to a rule. As part of the GetEventPrediction response, Amazon Fraud Detector will return the defined outcomes for any matched rule.

In [ ]:
# Collect existing outcomes
response = fraud_detector.get_outcomes()
existing_outcome_names = [ outcome['name'] for outcome in response['outcomes'] ]


OUTCOME_BLOCK = 'block'
OUTCOME_REVIEW = 'review'
OUTCOME_FRICTION = 'friction'
OUTCOME_APPROVE = 'approve'

if OUTCOME_BLOCK not in existing_outcome_names:
    fraud_detector.put_outcome(
        name = OUTCOME_BLOCK,
        description = 'this outcome blocks the event'
    )

if OUTCOME_REVIEW not in existing_outcome_names:
    fraud_detector.put_outcome(
        name = OUTCOME_REVIEW,
        description = 'this outcome sidelines event for review'
    )

if OUTCOME_FRICTION not in existing_outcome_names:
    fraud_detector.put_outcome(
        name = OUTCOME_FRICTION,
        description = 'this outcome frictions the event'
    )

if OUTCOME_APPROVE not in existing_outcome_names:
    fraud_detector.put_outcome(
        name = OUTCOME_APPROVE,
        description = 'this outcome approves the event'
    )

##### Create Rules
A rule is a condition that tells Amazon Fraud Detector how to interpret variable values during a fraud prediction. A rule consists of one or more variables, a logic expression, and one or more outcomes. A detector must have at least one associated rule. Rules in a detector are evaluated as part of a fraud prediction.

In [ ]:
class Rule:
    
    def __init__(self, id, detector_id, expression, outcomes):
        self.id = id
        self.detector_id = detector_id
        self.expression = expression
        self.outcomes = outcomes
        self.rule_version = None
    
    def may_create_rule_and_collect_rule_version(self, fraud_detector):
        """
        Assume the rule comparison is based on rule id and related expression only. 
        """
        response = fraud_detector.get_rules(
            ruleId = self.id,
            detectorId = self.detector_id
        )
        to_create_rule = ( not response['ruleDetails'] ) or ( response['ruleDetails'][0]['expression'] != self.expression )
        if not response['ruleDetails']:
            result = fraud_detector.create_rule(
                ruleId = self.id,
                detectorId = self.detector_id,
                expression = self.expression,
                language = 'DETECTORPL',
                outcomes = self.outcomes
            )
            self.rule_version = result['rule']['ruleVersion']
        elif response['ruleDetails'][0]['expression'] != self.expression:
            result = fraud_detector.update_rule_version(
                rule={
                    'detectorId': self.detector_id,
                    'ruleId': self.id,
                    'ruleVersion': response['ruleDetails'][0]['ruleVersion']
                },
                expression = self.expression,
                language = 'DETECTORPL',
                outcomes = self.outcomes
            )
            self.rule_version = result['rule']['ruleVersion']
        else:
            self.rule_version = response['ruleDetails'][0]['ruleVersion']

    def to_detector_version_output(self):
        """
        An rule output for detector version creation.
        """
        return {
            'detectorId' : self.detector_id,
            'ruleId' : self.id,
            'ruleVersion' : self.rule_version
        }

In [ ]:
RULE_ID_BLOCK = 'rule_block'
RULE_ID_REVIEW = 'rule_review'
RULE_ID_FRICTION = 'rule_friction'
RULE_ID_APPROVE = 'rule_approve'

# model insightscore expression is base on "{model_name}_insightscore"
rule_block = Rule(RULE_ID_BLOCK, DETECTOR_ID, f"${MODEL_ID}_insightscore >= 950", [ OUTCOME_BLOCK ] )
rule_review = Rule(RULE_ID_REVIEW, DETECTOR_ID, f"${MODEL_ID}_insightscore >= 855 and ${MODEL_ID}_insightscore < 950", [ OUTCOME_REVIEW ] )
rule_friction = Rule(RULE_ID_FRICTION, DETECTOR_ID, f"${MODEL_ID}_insightscore >= 600 and ${MODEL_ID}_insightscore < 855", [ OUTCOME_FRICTION ] )
rule_approve = Rule(RULE_ID_APPROVE, DETECTOR_ID, f"${MODEL_ID}_insightscore < 600", [ OUTCOME_APPROVE ] )

rule_block.may_create_rule_and_collect_rule_version(fraud_detector)
rule_review.may_create_rule_and_collect_rule_version(fraud_detector)
rule_friction.may_create_rule_and_collect_rule_version(fraud_detector)
rule_approve.may_create_rule_and_collect_rule_version(fraud_detector)

##### Create Detector Version
A detector version defines the specific models and rules that will be run as part of the GetEventPrediction request. You can add any of the rules defined within a detector to the detector version. You can also add any model trained on the evaluated event type.

Each detector version has a status of DRAFT, ACTIVE, or INACTIVE. Only one detector version can be in ACTIVE status at a time. During the GetEventPrediction request, Amazon Fraud Detector will use the ACTIVE detector if no DetectorVersion is specified.

In [ ]:
def create_detector_version(fraud_detector, detectorId, rule_arr, model_id, model_version_number, ruleExecutionMode = 'FIRST_MATCHED'):
    rules = [rule.to_detector_version_output() for rule in rule_arr]
    response = fraud_detector.create_detector_version(
        detectorId = detectorId,
        rules = rules,
        modelVersions = [{
            'modelId' : model_id,
            'modelType': 'ONLINE_FRAUD_INSIGHTS',
            'modelVersionNumber' : model_version_number
        }],
        ruleExecutionMode = ruleExecutionMode
    )

    return response['detectorVersionId']

def update_detector_version_status(fraud_detector, detector_id, detector_version_id, status = 'ACTIVE'):
    response = fraud_detector.update_detector_version_status(
        detectorId = detector_id,
        detectorVersionId = detector_version_id,
        status = status
    )
    return response['ResponseMetadata']['HTTPStatusCode'] == 200

In [ ]:
# create detector version
detector_version_id = create_detector_version(
    fraud_detector, 
    DETECTOR_ID, 
    [rule_block, rule_review, rule_friction, rule_approve],
    MODEL_ID,
    model_version_number
)
result = update_detector_version_status(
    fraud_detector, 
    DETECTOR_ID,
    detector_version_id
)

##### Inference
-----
The following applies the get_event_prediction endpoint to your recrods in your data frame.    

<i> Note: this uses the Dask backend to parallelize the prediction calls. </i>

<div class="alert alert-info"> 💡 <strong> get_event_prediction </strong>

To specify the number of records to score you change the record_count to a specific number (e.g., if you want to just predict on say 100 records). By default it assumes you want to apply predictions to the whole dataset. Once completed convert json to a pandas dataframe, and appends any existing labels, and analyze based on score threshold for a particular false positive rate (FPR).

</div>

this is all you need to run predictions: 

```python

client.get_event_prediction(detectorId=DETECTOR_NAME, 
                            detectorVersionId=DETECTOR_VERSION,
                            eventId = '222222',
                            eventTypeName = EVENT_TYPE,
                            eventTimestamp = '2020-07-27 12:01:01', 
                            entities = [{'entityType': ENTITY_TYPE, 'entityId':'11111'}],
                            eventVariables=  record)
```

Example of what a record would look like: 

```python
record = [{'order_amt': '8036.0',
  'ip_address': '192.18.59.93',
  'email_address': 'synth_george_hayduke@example.com',
  'cc_bin': '42785',
  'billing_postal': '17740-2745',
  'shipping_postal': '20950-6945',
  'customer_name': 'Geroge Hayduke'}]
```

In [ ]:
@dask.delayed
def _predict(fraud_detector, record, detector_id, detector_version_id, model_id, event_type_name, entity_type_name, event_timestamp):
    eventId = uuid.uuid1()
    try:
        pred = fraud_detector.get_event_prediction(
            detectorId = detector_id, 
            detectorVersionId = detector_version_id,
            eventId = str(eventId),
            eventTypeName = event_type_name,
            eventTimestamp = event_timestamp, 
            entities = [{'entityType': entity_type_name, 'entityId':str(eventId.int)}],
            eventVariables =  record) 
        
        record["score"]   = pred['modelScores'][0]['scores']["{0}_insightscore".format(model_id)]
        record["outcomes"]= pred['ruleResults'][0]['outcomes']
        return record
    
    except:
        record["score"]   = "-999"
        record["outcomes"]= "error"
        return record

def do_predictions(fraud_detector, df, detector_id, detector_version_id, model_id, event_type_name, entity_type_name):
    start = time.time()
    predict_score = []
    event_timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
    
    predict_data  = df.head(record_count).astype(str).to_dict(orient='records')
    i=0
    for record in predict_data:
        clear_output(wait=True)
        rec = dask.delayed(_predict)(fraud_detector, record, detector_id, detector_version_id, model_id, event_type_name, entity_type_name, event_timestamp)
        predict_score.append(rec)
        i += 1
        print("current progress: ", round((i/record_count)*100,2), "%" )


    predict_recs = dask.compute(*predict_score)

    # Calculate time taken and print results
    time_taken = time.time() - start
    tps = len(predict_recs) / time_taken

    print ('Process took %0.2f seconds' %time_taken)
    print ('Scored %d records' %len(predict_recs))

    return predict_recs

def plot_predictions(predictions, threshold = 950):
    predictions['score_bin'] = pd.cut(predictions['score'],100 ,labels = list(range(0,1000,10)))
    predictions.loc[predictions['score'] >= threshold, 'score_outcome'] = 'predicted fraud'
    predictions.loc[predictions['score'] < threshold, 'score_outcome'] = 'predicted legit'
    ctab = pd.crosstab(predictions['score_bin'], predictions['score_outcome']).reset_index()

    plt.figure(figsize=(20,10))
    plt.bar(ctab['score_bin'], ctab['predicted fraud'], width=7)
    plt.bar(ctab['score_bin'], ctab['predicted legit'], width=7)
    plt.axvline(x = threshold, ls='--', C='red')
    plt.title("Sample Data, Score Distribution")
    plt.text( threshold, 16, f'score >= {threshold}')
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.show()    

In [ ]:
# Read the evaluation data set
df = pd.read_csv("./project_1_newaccounts_5k.csv")
df['user_agent'] = df['user_agent'].str.slice(start=0, stop=254)
df.head()

In [ ]:
# record_count = df.shape[0] # -- override  to run all records in file 
record_count = 100
predict_recs = do_predictions(
    fraud_detector, 
    df[event_variables], 
    DETECTOR_ID, 
    detector_version_id, 
    MODEL_ID, 
    EVENT_TYPE_NAME, 
    ENTITY_TYPE_NAME)
predictions = pd.DataFrame.from_dict(predict_recs, orient='columns')
predictions.head(10)

In [ ]:
plot_predictions(predictions)

##### Use different threshold

Let's try to do threshold = 985 and plot the predictions

In [ ]:
THRESHOLD = 985

# Update rules with new threshold
rule_block = Rule(RULE_ID_BLOCK, DETECTOR_ID, f"${MODEL_ID}_insightscore >= {THRESHOLD}", [ OUTCOME_BLOCK ] )
rule_block.may_create_rule_and_collect_rule_version(fraud_detector)

rule_review = Rule(RULE_ID_REVIEW, DETECTOR_ID, f"${MODEL_ID}_insightscore >= 855 and ${MODEL_ID}_insightscore < {THRESHOLD}", [ OUTCOME_REVIEW ] )
rule_review.may_create_rule_and_collect_rule_version(fraud_detector)

detector_version_id = create_detector_version(
    fraud_detector, 
    DETECTOR_ID, 
    [rule_block, rule_review, rule_friction, rule_approve],
    MODEL_ID,
    model_version_number
)
result = update_detector_version_status(
    fraud_detector, 
    DETECTOR_ID,
    detector_version_id
)

In [ ]:
# record_count = df.shape[0] # -- override  to run all records in file 
record_count = 100
predict_recs = do_predictions(
    fraud_detector, 
    df[event_variables], 
    DETECTOR_ID, 
    detector_version_id, 
    MODEL_ID, 
    EVENT_TYPE_NAME, 
    ENTITY_TYPE_NAME)
predictions = pd.DataFrame.from_dict(predict_recs, orient='columns')
predictions.head(10)

In [ ]:
plot_predictions(predictions, THRESHOLD)

##### Delete resources

In [ ]:
# delete detector versions, rules and detector
response = fraud_detector.describe_detector(detectorId = DETECTOR_ID)
for detector_summary in response['detectorVersionSummaries']:
    if detector_summary['status'] == 'ACTIVE':
        update_detector_version_status(
            fraud_detector, 
            DETECTOR_ID,
            detector_summary['detectorVersionId'],
            'INACTIVE'
        )
    result = fraud_detector.delete_detector_version(
        detectorId = DETECTOR_ID,
        detectorVersionId = detector_summary['detectorVersionId']
    )

result = fraud_detector.get_rules( detectorId = DETECTOR_ID )
for rule in result['ruleDetails']:
    response = fraud_detector.delete_rule(
        rule={
            'detectorId': rule['detectorId'],
            'ruleId': rule['ruleId'],
            'ruleVersion': rule['ruleVersion']
        }
    )
    
fraud_detector.delete_detector(detectorId = DETECTOR_ID)    

In [ ]:
# undeploy activate model version (model version and model can't be deleted)
active_model_version = get_active_model_version(MODEL_ID)
if active_model_version:
    fraud_detector.update_model_version_status (
        modelId = active_model_version['modelId'],
        modelType = 'ONLINE_FRAUD_INSIGHTS',
        modelVersionNumber = active_model_version['modelVersionNumber'],
        status = 'INACTIVE'
    )